# Zomato Geospatial Analysis

In [1]:
!pip install folium==0.10.0

     |████████████████████████████████| 92kB 5.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.10.0 which is incompatible.
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


Importing the Libraries

In [2]:
#importing the basic standard libraries for Data Science
import pandas as pd
import numpy as np
import seaborn as sns


Working on the Data

In [3]:
# mounting
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [4]:
# Reading the data
df=pd.read_csv("/content/drive/My Drive/Deep Learning Course/zomato.csv")

In [5]:
# Checking the head of the dataset
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [6]:
# Checking the number of Null values in the data set
df.isna().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

In [7]:
# Number of Null Values in the Location Column
df['location'].isna().value_counts()

False    51696
True        21
Name: location, dtype: int64

In [8]:
# Locations along with the total number of unique locations
print(df['location'].unique())
print("Number of Unique Values is Location",df['location'].nunique())

['Banashankari' 'Basavanagudi' 'Mysore Road' 'Jayanagar'
 'Kumaraswamy Layout' 'Rajarajeshwari Nagar' 'Vijay Nagar' 'Uttarahalli'
 'JP Nagar' 'South Bangalore' 'City Market' 'Nagarbhavi'
 'Bannerghatta Road' 'BTM' 'Kanakapura Road' 'Bommanahalli' nan
 'CV Raman Nagar' 'Electronic City' 'HSR' 'Marathahalli' 'Sarjapur Road'
 'Wilson Garden' 'Shanti Nagar' 'Koramangala 5th Block'
 'Koramangala 8th Block' 'Richmond Road' 'Koramangala 7th Block'
 'Jalahalli' 'Koramangala 4th Block' 'Bellandur' 'Whitefield'
 'East Bangalore' 'Old Airport Road' 'Indiranagar' 'Koramangala 1st Block'
 'Frazer Town' 'RT Nagar' 'MG Road' 'Brigade Road' 'Lavelle Road'
 'Church Street' 'Ulsoor' 'Residency Road' 'Shivajinagar' 'Infantry Road'
 'St. Marks Road' 'Cunningham Road' 'Race Course Road' 'Commercial Street'
 'Vasanth Nagar' 'HBR Layout' 'Domlur' 'Ejipura' 'Jeevan Bhima Nagar'
 'Old Madras Road' 'Malleshwaram' 'Seshadripuram' 'Kammanahalli'
 'Koramangala 6th Block' 'Majestic' 'Langford Town' 'Central Bangalo

In [9]:
# Removing the NULl values from the location column
df.dropna(axis=0,subset=['location'],inplace=True)

In [10]:
# Storing the unique locations in a variable
locations=df['location'].unique()

In [11]:
# Storing the locations in a data frame
data=pd.DataFrame()
# Creating a new column to store the unique locations
data['Name']=locations

In [12]:
data.head()

,Name
0,Banashankari
1,Basavanagudi
2,Mysore Road
3,Jayanagar
4,Kumaraswamy Layout


In [13]:
# Creating the Nominatim object from geopy
from geopy.geocoders import Nominatim
geolocator=Nominatim(user_agent='app')

In [14]:
# Finding the Latitude and Longitude
lat=[]
lon=[]
for locatio in data['Name']:
  # Using the geolocator to find the latitude and longitude
  locatio=geolocator.geocode(locatio) 
  if locatio is None:  
    lat.append(np.nan)
    lon.append(np.nan)
  else:
    lat.append(locatio.latitude)
    lon.append(locatio.longitude)

In [15]:
data['Lattitude']=lat
data['Longitude']=lon

In [16]:
data.head()

,Name,Lattitude,Longitude
0,Banashankari,15.887678,75.704678
1,Basavanagudi,12.941726,77.575502
2,Mysore Road,12.387214,76.666963
3,Jayanagar,27.643927,83.052805
4,Kumaraswamy Layout,12.908149,77.555318


In [17]:
# To make a geospatial analysis, count of location/restaurants is required
x=df['location'].value_counts().reset_index()
x.columns=['Name','Count']
# merging it into one single dataset
final=x.merge(data,on='Name',how='left').dropna()

In [18]:

# Head of the dataset
final['Longitude'].isna().value_counts()


False    92
Name: Longitude, dtype: int64

Generating the Map

In [19]:
# A function to generate the base map that accepts longitudes and start zoom level as parameters
def generatebasemap(default_locations=[12.97,77.59],default_zoom_start=12):
  basemap=folium.Map(location=default_locations,zoom_start=default_zoom_start)
  return basemap

In [20]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(15000)
print(sys.getrecursionlimit())


1000
15000


In [94]:
import folium
basemap=generatebasemap()
basemap

In [22]:
# Creating a heatmap for restaurant density
from folium.plugins import HeatMap
HeatMap(data=final[['Lattitude','Longitude','Count']],min_opacity=0.05,max_val=0.5,blur=15,alpha=0.5).add_to(basemap)

In [23]:
basemap

Marker and Cluster Analysis

In [25]:
# Here we will apply markers to various locations
from folium.plugins import FastMarkerCluster
FastMarkerCluster(data=final[['Lattitude','Longitude','Count']],min_opacity=0.05,max_val=0.5,blur=15,alpha=0.5).add_to(basemap)

In [26]:
# Here we can see the clusters along with their locations and markers
basemap

Cluster Analysis in Depth

In [27]:
# Here we will find the area with the maximum average rating
# The unique values given by customers as rating out of 5 are these
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', nan, '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [32]:
# Here we need to get rid of null values
df.dropna(axis=0,subset=['rate'],inplace=True)

In [34]:
def splitting(x):
 
 return x.split("/")[0]

In [35]:
# Now we need to split the values to get the rating out of 5
df['rating']=df['rate'].apply(splitting)

In [37]:
df['rate'].unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [39]:
# Here we replace the values where there is a new location with 0
df.replace(to_replace="NEW",value=0,inplace=True)

In [40]:
df.replace(to_replace="-",value=0,inplace=True)

In [41]:
df['rating'].unique()

array(['4.1', '3.8', '3.7', '3.6', '4.6', '4.0', '4.2', '3.9', '3.1',
       '3.0', '3.2', '3.3', '2.8', '4.4', '4.3', 0, '2.9', '3.5', '2.6',
       '3.8 ', '3.4', '4.5', '2.5', '2.7', '4.7', '2.4', '2.2', '2.3',
       '3.4 ', '3.6 ', '4.8', '3.9 ', '4.2 ', '4.0 ', '4.1 ', '3.7 ',
       '3.1 ', '2.9 ', '3.3 ', '2.8 ', '3.5 ', '2.7 ', '2.5 ', '3.2 ',
       '2.6 ', '4.5 ', '4.3 ', '4.4 ', '4.9', '2.1', '2.0', '1.8', '4.6 ',
       '4.9 ', '3.0 ', '4.8 ', '2.3 ', '4.7 ', '2.4 ', '2.1 ', '2.2 ',
       '2.0 ', '1.8 '], dtype=object)

In [42]:
# Converting all the object type to integer
df['rating']=pd.to_numeric(df['rating'])

In [50]:
# Here we group the restuaturants and their ratings by location and assign the average value
ratings=df.groupby('location')['rating'].mean().sort_values(ascending=False)

In [58]:
rated_locations=ratings.index
ratings_location=ratings.values

In [59]:
# Finding the Latitude and Longitude of the above locations
lat=[]
lon=[]
for locatio in rated_locations:
  # Using the geolocator to find the latitude and longitude
  locatio=geolocator.geocode(locatio) 
  if locatio is None:  
    lat.append(np.nan)
    lon.append(np.nan)
  else:
    lat.append(locatio.latitude)
    lon.append(locatio.longitude)

In [60]:
# Making a new Data Frame for ratings
final_ratings=pd.DataFrame()
final_ratings['Location']=rated_locations
final_ratings['Lattitude']=lat
final_ratings['Longitude']=lon
final_ratings['Ratings']=ratings_location

In [63]:
# Removing any Null values if exists
final_ratings.dropna(inplace=True)

In [64]:
HeatMap(data=final_ratings[['Lattitude','Longitude','Ratings']]).add_to(basemap)


In [65]:
basemap

Analysing North Indian Cuisines

In [66]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city),rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.1
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.1
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.8
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.7
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.8


In [85]:
# Finding Cuisines that provide North Indian
north_india=df[df['cuisines']=='North Indian']

In [86]:
# Finding Cuisines by location along with their count
north=north_india.groupby('location')['url'].count().reset_index()

In [87]:
# Merging the two dataframes to incorporate lattitudes and longitudes
north.columns=['Name','Count']
north=north.merge(data,on='Name',how='left')#.dropna(inplace=True)

In [95]:
# Making the heatmap
HeatMap(north[['Lattitude','Longitude','Count']]).add_to(basemap)
basemap


Generalising for all cuisines

In [111]:
def cuisine_map(cuisine):
  basemap=generatebasemap()
  basemap 
  filter=df['cuisines']==cuisine
  df2=df[filter]
  cuisine_data=df2.groupby('location')['url'].count().reset_index()
  cuisine_data.columns=['Name','Count']
  cuisine_data=cuisine_data.merge(data,on='Name',how='left')
  cuisine_data.dropna(inplace=True)
  HeatMap(cuisine_data[['Lattitude','Longitude','Count']]).add_to(basemap)
  FastMarkerCluster(cuisine_data[['Lattitude','Longitude','Count']]).add_to(basemap)
  return basemap

In [100]:
df['cuisines'].unique()

array(['North Indian, Mughlai, Chinese', 'Chinese, North Indian, Thai',
       'Cafe, Mexican, Italian', ..., 'Tibetan, Nepalese',
       'North Indian, Street Food, Biryani',
       'North Indian, Chinese, Arabian, Momos'], dtype=object)

In [112]:
cuisine_map("North Indian")